On cherche à prédire l'évolution de la méta (la répartition des decks des joueurs entres les différents archétypes); nous disposons de la répartition des joueurs et du pourcentage de victoire de chaque archétype contre chaque autre archétype individuellement et collectivement.

On n'a pour l'insant que des données sur les 10 decks les plus populaires chaque semaine.

In [74]:
import pandas as pd
import numpy as np

features = pd.read_excel('C:/Users/timot/Documents/Perso/MTG Datastore/MTG Data Kaldheim By Week.xlsx')
features_list =features.columns

features

,Week,% metagame,WINRATE vs. Metagame,vs Izzet\nTempo,vs Gruul\nAdventures,vs Dimir\nRogues,vs Naya\nAdventures,vs Abzan\nYorion,vs Rakdos\nMidrange,vs Mono-red\nAggro,...,vs Mono-red\nAggro.15,vs Boros\nAggro.15,vs Esper\nYorion.15,vs Mono-green\nFood.15,vs Ultimatum\nRamp.15,vs Jund\nMidrange.15,vs Dimir\nControl.15,vs Mono-green\nAggro.15,vs Mono-White\nControl.15,vs Other.15
0,1,8.4/100,62.1%,0.5,53.3%,9.1%,33.3%,71.4%,0.4,0.25,...,46.7%,37.5%,57.1%,0.35,18.2%,36.4%,53.8%,12.5%,22.2%,0.5
1,2,12.8/100,56.3%,0.5,68.3%,76.7%,46.7%,58.1%,56.3%,65.7%,...,43.1%,36.4%,0.5,42.9%,32.1%,57.7%,38.1%,0.4,53.3%,0.5
2,3,19.8/100,51.1%,0.5,55.5%,40.3%,0.48,60.2%,46.5%,0.43,...,55.3%,47.2%,36.8%,51.6%,57.9%,47.1%,52.4%,31.8%,37.5%,0.5
3,4,16.5/100,49.2%,0.5,38.5%,0.55,44.4%,36.8%,49.2%,55.4%,...,57.9%,47.9%,52.2%,51.6%,56.5%,17.6%,0.5,62.5%,41.7%,0.5
4,5,15.4/100,41.6%,0.5,40.5%,53.1%,23.5%,31.8%,0.32,42.9%,...,66.7%,0.75,0.4,0.5,0.75,0,0,-,0,0.5
5,6,17.4/100,53.2%,0.5,0.54,40.1%,44.7%,57.7%,42.5%,50.6%,...,38.5%,56.5%,48.9%,35.6%,39.5%,44.8%,33.3%,38.9%,44.4%,0.5
6,7,18.9/100,52.7%,0.5,0.44,0.5,0.41,61.9%,44.1%,64.7%,...,46.4%,44.4%,18.8%,35.3%,41.7%,33.3%,0.2,83.3%,46.2%,0.5
7,8,19.6/100,55.8%,0.5,48.9%,0.4,39.1%,47.8%,93.8%,46.2%,...,45.5%,31.6%,33.3%,0.4,-,0.25,66.7%,0.5,1,0.5
8,9,18.8/100,54.9%,0.5,0.55,58.5%,47.7%,59.8%,51.1%,0.58,...,51.7%,0.35,0.5,62.5%,53.8%,0.6,0.5,0.75,33.3%,0.5
9,10,15.8/100,50.8%,0.5,40.9%,55.1%,70.2%,50.6%,43.8%,38.7%,...,42.1%,44.4%,48.8%,44.2%,31.8%,61.1%,38.1%,41.2%,26.7%,0.5


In [75]:
features = features.fillna(0)  #remplaçage des NaN par 0

#on remplace les '%' et les '/100' par une division par 100 et les '-' par des 0
for i in range(len(features_list)):
    for j in range(len(features[features_list[0]])):
        if type(features[features_list[i]][j]) == str:
            if features[features_list[i]][j].find('%') != -1:
                features[features_list[i]][j] = float(features[features_list[i]][j].replace('%', ''))/100
            elif features[features_list[i]][j].find('/100') != -1:
                features[features_list[i]][j] = float(features[features_list[i]][j].replace('/100', ''))/100
            else :
                features[features_list[i]][j] = float(features[features_list[i]][j].replace('-', '0'))
#print(features)

#Les colonnes '% Metagame', qui sont celles que nous voulons prédire, sont séparées par 17 colonnes de VS
#On va donc les isoler

meta_list = [features_list[i*18+1] for i in range(np.size(features_list)//18)]
print(meta_list) #les quantités à prédire

labels = np.array(features[meta_list]) #les quantités à prédire
labels = np.delete(labels, (0), axis = 0) #au temps 0, on n'a pas encore fait de prédiction

features = np.array(features) #le format array est nécessaire pour l'utilisation de RandomForest
last_line = features[9,:] #->le but : prédire l'état de la méta la semaine prochaine à partir de la dernière ligne
features = np.delete(features, (9), axis=0) #au dernier instant, on n'a pas de prédictions pour le modèle

#last_line = features[8,:]
#features = np.delete(features, (8), axis=0)

['% metagame', '% metagame.1', '% metagame.2', '% metagame.3', '% metagame.4', '% metagame.5', '% metagame.6', '% metagame.7', '% metagame.8', '% metagame.9', '% metagame.10', '% metagame.11', '% metagame.12', '% metagame.13', '% metagame.14', '% metagame.15']


<ipython-input-75-34659e315110>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[features_list[i]][j] = float(features[features_list[i]][j].replace('/100', ''))/100
<ipython-input-75-34659e315110>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[features_list[i]][j] = float(features[features_list[i]][j].replace('%', ''))/100
<ipython-input-75-34659e315110>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[fe

Dans la version actuelle, les 10 archétypes les plus populaires changent presque toutes les semaines, de même que le classement des archétypes qui restent d'une semaine sur l'autre.
La version actuelle du code "anonymise" les archétypes : on se contente de prendre en compte la représentation de "Archétype n°1", "Archétype n°2",...

In [76]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25)  #,
                                                                           #random_state = 35)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (6, 289)
Training Labels Shape: (6, 16)
Testing Features Shape: (3, 289)
Testing Labels Shape: (3, 16)


In [77]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators= 1000, random_state=35)
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=35)

In [78]:
predictions = rf.predict(test_features)
errors = abs(predictions - test_labels)
print('Mean Absolute Error:', round(np.mean(errors), 5))

Mean Absolute Error: 0.01177


In [79]:
ref_line = [] #Les représentation des archétypes à la dernière semaine, pour comparaison.
for i in range(np.size(features_list)//18):
    ref_line = ref_line + [last_line[i*18+1]]
last_line = last_line.reshape(1,-1)
predict_line = rf.predict(last_line)[0]

In [81]:
ref_line = [x*100 for x in ref_line]
predict_line = [x*100/sum(predict_line) for x in predict_line]
pd.DataFrame([ref_line,predict_line], columns=meta_list)

,% metagame,% metagame.1,% metagame.2,% metagame.3,% metagame.4,% metagame.5,% metagame.6,% metagame.7,% metagame.8,% metagame.9,% metagame.10,% metagame.11,% metagame.12,% metagame.13,% metagame.14,% metagame.15
0,15.800000,13.800000,12.200000,10.300000,4.700000,4.700000,4.400000,4.300000,4.000000,3.700000,3.400000,1.900000,1.800000,1.600000,1.600000,11.800000
1,18.072584,14.691365,11.472775,9.140397,5.898317,5.105064,4.290921,3.839718,3.502866,2.690522,2.143561,1.682363,1.545523,1.413781,1.341912,13.168332
